In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'XOM'
start_date = '2020-01-01'
end_date = '2024-01-01'
1
# Hyperparameters
n_steps = 2
window_size = 5
step_size = 2
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.00005
train_ratio = 0.9
n_units = 16
batch_size = 4

# If you change features or other logic, do so below
if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")


def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Next day prediction setup
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    # Simple features
    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates


def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates


def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)


def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    # We'll compile after loading weights if needed
    return model


def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape


def rolling_training_early_stopping(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for {ticker}. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    # Just run with the current top-level defined parameters:
    rolling_training_early_stopping(ticker, window_size, step_size, n_steps)


[*********************100%%**********************]  1 of 1 completed
[INFO] Training initial window with Early Stopping...
[RESULT] Initial window validation MAPE: 0.51%

[INFO] Iteration 1: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.19%
[INFO] Predicted last sample of window: Actual=69.80, Pred=69.67, MAPE=0.19%

[INFO] Iteration 2: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.10%
[INFO] Predicted last sample of window: Actual=69.09, Pred=69.02, MAPE=0.10%

[INFO] Iteration 3: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.71%
[INFO] Predicted last sample of window: Actual=68.56, Pred=69.04, MAPE=0.71%

[INFO] Iteration 4: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.07%
[INFO] Predicted last sample of window: Actual=67.19, Pred=67.91, MAPE=1.07%

[INFO] Iteration 5: training on new window with Early Stopping...
[RESULT] Window validation MAPE

[RESULT] Window validation MAPE: 0.90%
[INFO] Predicted last sample of window: Actual=43.94, Pred=43.55, MAPE=0.90%

[INFO] Iteration 38: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 7.40%
[INFO] Predicted last sample of window: Actual=47.46, Pred=43.95, MAPE=7.40%

[INFO] Iteration 39: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 5.09%
[INFO] Predicted last sample of window: Actual=43.14, Pred=45.34, MAPE=5.09%

[INFO] Iteration 40: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.39%
[INFO] Predicted last sample of window: Actual=44.83, Pred=44.66, MAPE=0.39%

[INFO] Iteration 41: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.12%
[INFO] Predicted last sample of window: Actual=44.24, Pred=44.19, MAPE=0.12%

[INFO] Iteration 42: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.93%
[INFO] Predicted last sample of window: A

[RESULT] Window validation MAPE: 1.35%
[INFO] Predicted last sample of window: Actual=39.19, Pred=39.72, MAPE=1.35%

[INFO] Iteration 83: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.27%
[INFO] Predicted last sample of window: Actual=39.08, Pred=39.19, MAPE=0.27%

[INFO] Iteration 84: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.92%
[INFO] Predicted last sample of window: Actual=37.95, Pred=38.30, MAPE=0.92%

[INFO] Iteration 85: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.57%
[INFO] Predicted last sample of window: Actual=36.90, Pred=37.85, MAPE=2.57%

[INFO] Iteration 86: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.66%
[INFO] Predicted last sample of window: Actual=36.27, Pred=36.87, MAPE=1.66%

[INFO] Iteration 87: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.63%
[INFO] Predicted last sample of window: A

[RESULT] Window validation MAPE: 4.62%
[INFO] Predicted last sample of window: Actual=47.88, Pred=45.67, MAPE=4.62%

[INFO] Iteration 128: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 4.53%
[INFO] Predicted last sample of window: Actual=50.31, Pred=48.03, MAPE=4.53%

[INFO] Iteration 129: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.63%
[INFO] Predicted last sample of window: Actual=48.84, Pred=48.53, MAPE=0.63%

[INFO] Iteration 130: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.62%
[INFO] Predicted last sample of window: Actual=48.11, Pred=48.89, MAPE=1.62%

[INFO] Iteration 131: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.77%
[INFO] Predicted last sample of window: Actual=46.90, Pred=47.73, MAPE=1.77%

[INFO] Iteration 132: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.11%
[INFO] Predicted last sample of wind

[RESULT] Window validation MAPE: 0.08%
[INFO] Predicted last sample of window: Actual=58.92, Pred=58.87, MAPE=0.08%

[INFO] Iteration 173: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.12%
[INFO] Predicted last sample of window: Actual=58.26, Pred=58.91, MAPE=1.12%

[INFO] Iteration 174: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.60%
[INFO] Predicted last sample of window: Actual=58.56, Pred=58.91, MAPE=0.60%

[INFO] Iteration 175: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.27%
[INFO] Predicted last sample of window: Actual=60.46, Pred=58.48, MAPE=3.27%

[INFO] Iteration 176: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.01%
[INFO] Predicted last sample of window: Actual=61.18, Pred=61.17, MAPE=0.01%

[INFO] Iteration 177: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.02%
[INFO] Predicted last sample of wind

...

[RESULT] Window validation MAPE: 4.74%
[INFO] Predicted last sample of window: Actual=91.39, Pred=95.72, MAPE=4.74%

[INFO] Iteration 308: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.88%
[INFO] Predicted last sample of window: Actual=91.48, Pred=90.67, MAPE=0.88%

[INFO] Iteration 309: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.50%
[INFO] Predicted last sample of window: Actual=85.21, Pred=87.34, MAPE=2.50%

[INFO] Iteration 310: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.55%
[INFO] Predicted last sample of window: Actual=89.03, Pred=87.65, MAPE=1.55%

[INFO] Iteration 311: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.87%
[INFO] Predicted last sample of window: Actual=88.12, Pred=89.77, MAPE=1.87%

[INFO] Iteration 312: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.32%
[INFO] Predicted last sample of wind

[RESULT] Window validation MAPE: 0.15%
[INFO] Predicted last sample of window: Actual=105.88, Pred=106.04, MAPE=0.15%

[INFO] Iteration 353: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.86%
[INFO] Predicted last sample of window: Actual=107.55, Pred=106.62, MAPE=0.86%

[INFO] Iteration 354: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.54%
[INFO] Predicted last sample of window: Actual=110.81, Pred=109.11, MAPE=1.54%

[INFO] Iteration 355: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.43%
[INFO] Predicted last sample of window: Actual=109.61, Pred=111.18, MAPE=1.43%

[INFO] Iteration 356: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.19%
[INFO] Predicted last sample of window: Actual=112.31, Pred=110.98, MAPE=1.19%

[INFO] Iteration 357: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.22%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 1.68%
[INFO] Predicted last sample of window: Actual=112.81, Pred=110.91, MAPE=1.68%

[INFO] Iteration 397: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.19%
[INFO] Predicted last sample of window: Actual=111.61, Pred=111.82, MAPE=0.19%

[INFO] Iteration 398: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.98%
[INFO] Predicted last sample of window: Actual=109.13, Pred=111.30, MAPE=1.98%

[INFO] Iteration 399: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.21%
[INFO] Predicted last sample of window: Actual=106.54, Pred=108.89, MAPE=2.21%

[INFO] Iteration 400: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 5.16%
[INFO] Predicted last sample of window: Actual=101.62, Pred=106.86, MAPE=5.16%

[INFO] Iteration 401: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.41%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 1.39%
[INFO] Predicted last sample of window: Actual=105.97, Pred=104.49, MAPE=1.39%

[INFO] Iteration 441: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.27%
[INFO] Predicted last sample of window: Actual=104.54, Pred=104.82, MAPE=0.27%

[INFO] Iteration 442: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.58%
[INFO] Predicted last sample of window: Actual=101.38, Pred=104.00, MAPE=2.58%

[INFO] Iteration 443: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.29%
[INFO] Predicted last sample of window: Actual=101.62, Pred=101.33, MAPE=0.29%

[INFO] Iteration 444: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.05%
[INFO] Predicted last sample of window: Actual=103.89, Pred=101.76, MAPE=2.05%

[INFO] Iteration 445: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.56%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 1.18%
[INFO] Predicted last sample of window: Actual=104.84, Pred=103.60, MAPE=1.18%

[INFO] Iteration 485: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.15%
[INFO] Predicted last sample of window: Actual=103.66, Pred=103.82, MAPE=0.15%

[INFO] Iteration 486: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.65%
[INFO] Predicted last sample of window: Actual=104.96, Pred=104.27, MAPE=0.65%

[INFO] Iteration 487: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.15%
[INFO] Predicted last sample of window: Actual=104.46, Pred=104.62, MAPE=0.15%

[INFO] Iteration 488: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.08%
[INFO] Predicted last sample of window: Actual=104.57, Pred=104.49, MAPE=0.08%

[INFO] Iteration 489: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.06%
[INFO] Predicted last samp